In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [6]:
rows = rdd.take(10)
row = rows[0]

In [7]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 4, 5, 54), PULocationID=129, total_amount=8.51)

In [8]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [9]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [10]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(10)

[((datetime.datetime(2020, 1, 15, 13, 0), 39), (171.7, 6)),
 ((datetime.datetime(2020, 1, 3, 0, 0), 41), (224.6400000000001, 13)),
 ((datetime.datetime(2020, 1, 24, 18, 0), 52), (215.75, 17)),
 ((datetime.datetime(2020, 1, 3, 19, 0), 181), (171.49, 15)),
 ((datetime.datetime(2020, 1, 5, 8, 0), 166), (81.36, 4)),
 ((datetime.datetime(2020, 1, 4, 10, 0), 40), (48.25999999999999, 4)),
 ((datetime.datetime(2020, 1, 19, 11, 0), 42), (311.60000000000014, 27)),
 ((datetime.datetime(2020, 1, 5, 15, 0), 17), (135.96, 7)),
 ((datetime.datetime(2020, 1, 27, 22, 0), 41), (185.15000000000006, 17)),
 ((datetime.datetime(2020, 1, 10, 12, 0), 65), (334.11, 17))]

In [11]:
from collections import namedtuple

In [12]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [13]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [14]:
from pyspark.sql import types

In [15]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [16]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [17]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-15 13:00:00|  39|             171.7|    6|
|2020-01-03 00:00:00|  41| 224.6400000000001|   13|
|2020-01-24 18:00:00|  52|            215.75|   17|
|2020-01-03 19:00:00| 181|            171.49|   15|
|2020-01-05 08:00:00| 166|             81.36|    4|
|2020-01-04 10:00:00|  40| 48.25999999999999|    4|
|2020-01-19 11:00:00|  42|311.60000000000014|   27|
|2020-01-05 15:00:00|  17|            135.96|    7|
|2020-01-27 22:00:00|  41|185.15000000000006|   17|
|2020-01-10 12:00:00|  65|            334.11|   17|
|2020-01-07 14:00:00| 216| 66.22999999999999|    3|
|2020-01-06 07:00:00| 130|            196.45|    8|
|2020-01-24 21:00:00| 116|218.47000000000006|   15|
|2020-01-22 16:00:00| 166| 783.7999999999996|   49|
|2020-01-14 22:00:00| 226|             23.85|    2|
|2020-01-14 15:00:00|  75| 1426.369999999999|   87|
|2020-01-04 

In [18]:
df_result.write.parquet('tmp/green-revenue')

In [19]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [20]:
duration_rdd

MapPartitionsRDD[31] at javaToPython at NativeMethodAccessorImpl.java:0

In [21]:
import pandas as pd

In [22]:
rows = duration_rdd.take(10)

In [23]:
df = pd.DataFrame(rows, columns=columns)

In [24]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [25]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [26]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [27]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [28]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
| 4.050000000000001|
|             13.45|
|             65.55|
|10.649999999999999|
|              4.45|
|               4.4|
|             11.25|
|              4.55|
|             13.45|
|             26.75|
|               2.9|
|             39.15|
|               2.5|
|               4.5|
|119.39999999999999|
|               9.8|
|               0.0|
|7.1499999999999995|
|              4.25|
|             12.05|
+------------------+
only showing top 20 rows

